## 05 RESNET

In [1]:
import torch
from PIL import Image
import pickle
from torchvision import models, transforms
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from sklearn.model_selection import train_test_split
import numpy as np
from torch.utils.data import Dataset, DataLoader
import time
import copy

D:\PROJEKAT_CLFO\venv\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'D:\PROJEKAT_CLFO\venv\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
name = 'nochers_5x143' #enter name of the pickle file
with open(name, 'rb') as f:
    image_paths = pickle.load(f)

In [4]:
model = models.resnet18(pretrained = True)

#model preformulation - we have 5 classes
model.fc = nn.Linear(512,5)

#CPU to GPU if available
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

#loss
criterion = nn.CrossEntropyLoss()

#optimizer
optimizer = optim.SGD(model.parameters(), lr = 0.001, momentum = 0.9)

#decay lr by 0.1 for every 7 epochs
schedular = lr_scheduler.StepLR(optimizer, step_size = 7, gamma = 0.1)

In [5]:
preprocess = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [6]:
categories = 5
samples_per_category = len(image_paths[0])
data = torch.zeros((samples_per_category*categories, 3, 224, 224))
labels = []

for i in range(samples_per_category):
    for j in range(categories):
        path = image_paths[j][i]
        input_image = Image.open(path)
        inx = i*categories+j
        if inx % 50 == 0 and j == 0:
            print(f'{inx}/{categories*samples_per_category}')
        try:
            input_tensor = preprocess(input_image)
            data[inx] = input_tensor
        except:
            print(inx, path)
            print('Mistake')
        labels.append(j)

0/715
10/715
20/715
30/715
40/715
50/715
60/715
70/715
80/715
90/715
100/715
110/715
120/715
130/715
140/715
150/715
160/715
170/715
180/715
190/715
200/715
210/715
220/715
230/715
240/715
250/715
260/715
270/715
280/715
290/715
300/715
310/715
320/715
330/715
340/715
350/715
360/715
370/715
380/715
390/715
400/715
410/715
420/715
430/715
440/715
450/715
460/715
470/715
480/715
490/715
500/715
510/715
520/715
530/715
540/715
550/715
560/715
570/715
580/715
590/715
600/715
610/715
620/715
630/715
640/715
650/715
660/715
670/715
680/715
690/715
700/715
710/715


In [7]:
labels = np.array(labels)

In [8]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size = 0.2)

In [9]:
class Data(Dataset):
    
    def __init__(self, data, labels):
        self.x = data
        self.y = torch.from_numpy(labels)
        self.samples = data.shape[0]
        self.features = data.shape[1]
        
    def __getitem__(self, index):
        # define indexing
        return self.x[index], self.y[index]
        
    def __len__(self):
        # len(dataset)
        return self.samples

In [10]:
train = Data(X_train, y_train)
test = Data(X_test, y_test)
print(f'train: {len(train)}, test: {len(test)}')
train.y.size()

train: 572, test: 143


torch.Size([572])

In [11]:
datasets = {'train': train, 'test': test}
dataloaders = {x: DataLoader(datasets[x], batch_size=5, num_workers=0) for x in ['train', 'test']}
dataset_sizes = {x: len(datasets[x]) for x in ['train', 'test']}
class_names = np.unique(labels)

In [20]:
print(dataset_sizes)
class_names

{'train': 572, 'test': 143}


array([0, 1, 2, 3, 4])

In [17]:
def train_model(model, criterion, optimizer, scheduler, num_epochs = 10):
    since = time.time()
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'test']:
            print(f'phase: {phase}')
            if phase == 'train':
                model.train()
            else:
                model.eval()
            
            running_loss = 0.0
            running_corrects = 0

            batch_number=0
            for inputs, labels in dataloaders[phase]:
                #if batch_number%5==0:
                    #print(f'batch munber: {batch_number}')
                batch_number+=1
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()
                
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels.long())

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'test' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model


In [18]:
train_model(model, criterion, optimizer, schedular, num_epochs = 10)

Epoch 1/10
----------
phase: train
train Loss: 0.6629 Acc: 0.7640
phase: test
test Loss: 1.6190 Acc: 0.4406

Epoch 2/10
----------
phase: train
train Loss: 0.4397 Acc: 0.8584
phase: test
test Loss: 1.9163 Acc: 0.3566

Epoch 3/10
----------
phase: train
train Loss: 0.4118 Acc: 0.8497
phase: test
test Loss: 1.8478 Acc: 0.3916

Epoch 4/10
----------
phase: train
train Loss: 0.3023 Acc: 0.8916
phase: test
test Loss: 2.3346 Acc: 0.3776

Epoch 5/10
----------
phase: train
train Loss: 0.2249 Acc: 0.9248
phase: test
test Loss: 1.9486 Acc: 0.4266

Epoch 6/10
----------
phase: train
train Loss: 0.0518 Acc: 0.9965
phase: test
test Loss: 1.9474 Acc: 0.4196

Epoch 7/10
----------
phase: train
train Loss: 0.0334 Acc: 1.0000
phase: test
test Loss: 1.9619 Acc: 0.4266

Epoch 8/10
----------
phase: train
train Loss: 0.0267 Acc: 1.0000
phase: test
test Loss: 1.9747 Acc: 0.4196

Epoch 9/10
----------
phase: train
train Loss: 0.0228 Acc: 1.0000
phase: test
test Loss: 1.9863 Acc: 0.4196

Epoch 10/10
-------